In [ ]:
import pandas as pd
import xgboost as xgb
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTENC
import matplotlib.pyplot as plt
from PIL import Image
from scipy.interpolate import BSpline, make_interp_spline, interp1d
#import rpy2.robjects as robjects
#from rpy2.robjects.packages import importr
import csv
from dfply import *
from xgboost import XGBClassifier
import itertools
import os
import logging
from glob import glob
import pickle

In [ ]:
def generate_corr(site, year):
    bt = pd.read_pickle('data/'+site+'/bt_'+site+'_'+str(year)+'.pkl')
    corr = bt.corr()
    corr.to_pickle('data/'+site+'/btcorr_'+site+'_'+str(year)+'.pkl')

In [ ]:
def calculate_corr_occurence2():
    files = []
    start_dir = os.getcwd()
    pattern   = "btcorr_*"
    for dir,_,_ in os.walk(start_dir):
        files.extend(glob(os.path.join(dir,pattern))) 

    meltcorrlist = list() 

    for file in files:
        site = file.split('/')[-1].split('.')[0].split('_')[1]
        year = file.split('/')[-1].split('.')[0].split('_')[2]
        corr = pd.read_pickle(file)
        corr = abs(corr)
        meltcorr = pd.melt(corr.reset_index(), id_vars=['index'])
        meltcorr = meltcorr[meltcorr['index'] != meltcorr['variable']]
        meltcorr['site'] = site
        meltcorr['year'] = year    
        meltcorrlist.append(meltcorr)

        
    meltcorrall = pd.concat(meltcorrlist)
    meltcorrall.columns = ['v1', 'v2', 'corr', 'site', 'year']
    meltcorrall.to_pickle('data/meltcorrall.pkl')
                              
#if __name__ == "__main__":
#    calculate_corr_occurence2()

In [ ]:
def calculate_corr_occurence(threshold = 0.5):
    files = []
    start_dir = os.getcwd()
    pattern   = "btcorr_*"
    for dir,_,_ in os.walk(start_dir):
        files.extend(glob(os.path.join(dir,pattern))) 

    meltcorrlist = list() 

    for file in files:
        site = file.split('/')[-1].split('.')[0].split('_')[1]
        year = file.split('/')[-1].split('.')[0].split('_')[2]
        corr = pd.read_pickle(file)
        meltcorr = pd.melt(corr.reset_index(), id_vars=['index'])
        meltcorr = meltcorr[meltcorr['value'] >= threshold]
        meltcorr = meltcorr[meltcorr['index'] != meltcorr['variable']]
        meltcorr['site'] = site
        meltcorr['year'] = year    
        meltcorrlist.append(meltcorr)

    meltcorrall = pd.concat(meltcorrlist)
    meltcorrall.columns = ['v1', 'v2', 'corr', 'site', 'year']
    meltcorrallcount = pd.melt(meltcorrall.drop('corr', axis=1), id_vars=['site', 'year']).drop('variable', axis=1).drop_duplicates().groupby('value').count().drop('site',axis=1).reset_index()
    meltcorrallcount.columns = ['value', 'count']
    meltcorrallcount.to_pickle('data/meltcorrallcount_'+str(threshold)+'.pkl')
    
if __name__ == "__main__":
    calculate_corr_occurence(threshold=0.5)

In [ ]:
def generate_corr_bt(site, year, threshold=0.5):   

    meltcorrallcount = pd.read_pickle('data/meltcorrallcount_'+str(threshold)+'.pkl')
    bt = pd.read_pickle('data/'+site+'/bt_'+site+'_'+str(year)+'.pkl')
    corr = bt.corr()
    corr.to_pickle('data/'+site+'/btcorr_'+site+'_'+str(year)+'.pkl')
    meltcorr = pd.melt(corr.reset_index(), id_vars=['index'])
    meltcorr = meltcorr[meltcorr['value'] >= threshold]
    meltcorr = meltcorr[meltcorr['index'] != meltcorr['variable']]
       
    correlated_group = list()
    correlated_group.append([meltcorr.iloc[0,0], meltcorr.iloc[0,1]])
    for i in range(meltcorr.shape[0]):
        newgroup = True
        for j in range(len(correlated_group)):
            if (meltcorr.iloc[i,0] in correlated_group[j] or meltcorr.iloc[i,1] in correlated_group[j]):
                correlated_group[j].append(meltcorr.iloc[i,0])
                correlated_group[j].append(meltcorr.iloc[i,1])
                newgroup = False
                break
        if newgroup:
            correlated_group.append([meltcorr.iloc[i,0], meltcorr.iloc[i,1]])

    keep_list = list()
    drop_list = list()

    for i in range(len(correlated_group)):
        correlated_groupX = list(set(correlated_group[i]))
        correlated_groupX.sort(reverse=True)
        maxcount = -1
        for feature in correlated_groupX:
            count = meltcorrallcount[meltcorrallcount['value'] == feature].iloc[0,1]
            if maxcount < count:
                maxfeature = feature
                maxcount = count
        keep_list.append(maxfeature)
        correlated_groupX.remove(maxfeature)
        drop_list.extend(correlated_groupX)
        
    with open('data/'+site+'/corrdropadd_'+site+'_'+str(year)+'.pkl', 'wb') as f:
        pickle.dump(drop_list, f)            
        pickle.dump(keep_list, f)            
    bt = bt.drop(drop_list, axis=1)
    bt.to_pickle('data/'+site+'/bt2_'+site+'_'+str(year)+'.pkl')